# Cart-Pole DQN

Sieć głębokiego Q-uczenia, wykorzystująca biblioteki Keras i OpenAI Gym, oparta na kodzie [Keona Kim'a](https://github.com/keon/deep-q-learning/blob/master/dqn.py).

#### Załadowanie zależności.

In [ ]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import os # Do tworzenia katalogów

#### Ustawienie hiperparametrów.

In [ ]:
env = gym.make('CartPole-v0') # Zainicjowanie środowiska.

In [ ]:
state_size = env.observation_space.shape[0]
state_size

4

In [ ]:
action_size = env.action_space.n
action_size

2

In [ ]:
batch_size = 32

In [ ]:
n_episodes = 1000 # Liczba gier, które ma odegrać agent.

In [ ]:
output_dir = 'model_output/cartpole/'

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#### Zdefiniowanie agenta.

In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) # Kolejka dwustronna; działa jak lista, ale elementy można dodawać / usuwać z dowolnego końca.
        self.gamma = 0.95 # Rozkład lub stopa dyskontowa: umożliwia agentowi uwzględnienie przyszłych działań oprócz natychmiastowych, ale dyskontowanych według tej stopy.
        self.epsilon = 1.0 # Współczynnik ekploracji, tj. stopień losowego działania. Więcej na początku niż później z powodu współczynnika epsilon_decay.
        self.epsilon_decay = 0.995 # Zmniejszenie ilości losowych ekploracji w miarę rosnącej (miejmy nadzieję) wydajności agenta w miarę upływu czasu.
        self.epsilon_min = 0.01 # Minimalny dozwolony odsetek  losowych ekploracji.
        self.learning_rate = 0.001 # Szybkość, z jaką sieć neuronowa dopasowuje parametry za pomocą gradientu prostego w celu zmniejszenia kosztu.
        self.model = self._build_model() # Metoda prywatna.
    
    def _build_model(self):
        # Sieć neuronowa aproksymująca funkcję Q-wartości.
        model = Sequential()
        model.add(Dense(32, activation='relu', 
                        input_dim=self.state_size)) # Pierwsza warstwa ukryta, wejściem są stany.
        model.add(Dense(32, activation='relu')) # Druga warstwa ukryta.
        model.add(Dense(self.action_size, activation='linear')) # Dwie czynności, więc dwa neurony: 0 i 1 (w lewo / w prawo).
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, 
                            reward, next_state, done)) # Lista poprzednich wyników, do wykorzystania przy powtórnym treningu.
        
    def train(self, batch_size): # Metoda trenująca sieć neuronową z wykorystaniem parametrów zapisanych w pamięci.
        minibatch = random.sample(self.memory, batch_size) # Minipacza zapisana w pamięci.
        for state, action, reward, next_state, done in minibatch: # Wyodrębnienie danych z każdej minipaczki.
            target = reward # Jeżeli koniec (wartość logiczna oznaczająca koniec gry, tj. osiągnięcie finalnego stanu), wtedy target = reward
            if not done: # Jeżeli nie koniec, prognozujemy przyszłą zdyskontowaną nagrodę.
                target = (reward + 
                          self.gamma * # (target) = reward + gamma * 
                          np.amax(self.model.predict(next_state)[0])) # (maksymalne docelowe Q bazujące na przyszłej czynności a')
            target_f = self.model.predict(state) # Aproksymowane skojarzenie bieżącego stanu z przyszłą zdyskontowaną nagrodą.
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0) # Pojedyncza epoka treningowa z x=state, y=target_f; trening zmniejsza stratę.
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def act(self, state):
        if np.random.rand() <= self.epsilon: # Wykonanie losowo wybranej czynności, jeżeli taki jest warunek.
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # Jeżeli czynność nie jest losowa, prognozujemy nagrodę na podstawie bieżącego stanu.
        return np.argmax(act_values[0]) # Wybranie czynności(w lewo lub w prawo) skutkującej uzyskaniem najwyższej nagrody.
    
    def save(self, name):
        self.model.save_weights(name)

    def load(self, name):
        self.model.load_weights(name)

#### Interakcja ze środowiskiem.

In [ ]:
agent = DQNAgent(state_size, action_size) # Zainicjowanie agenta.

In [ ]:
for e in range(n_episodes): # Iterowanie epizodów gry.
    
    state = env.reset() # Resetowanie stanu na początku każdego epizodu gry.
    state = np.reshape(state, [1, state_size])
    
    done = False
    time = 0 # Czas określa ramy epizodu. Celem jest jak najdłuższe utrzymanie tyczki w pionie.
    while not done: 
#         env.render()
        action = agent.act(state) # Czynnością jest ruch w lewo lub w prawo. Tu decydujemy.
        next_state, reward, done, _ = env.step(action) # Agent oddziałuje na środowisko i uzyskuje odpowiedź, 4 wartości, m.in. kąt odchylenia tyczki i położenie wózka.
        reward = reward if not done else -10 # Nagroda +1 za każdą dodatkową ramkę, gdy tyczka jest w pionie.
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done) # Zapamiętanie stanu (czynności, nagrody itp.) z poprzedniego kroku czasowego.
        state = next_state # Ustawienie "bieżącego stanu" dla iteracji jako bieżącego następnego stanu.
        if done: # Jeżeli to koniec epizodu:
            print("Epizod: {}/{}, ocena: {}, e: {:.2}" # Wyświetlenie oceny epizodu i wsp. epsilon agenta.
                  .format(e, n_episodes-1, time, agent.epsilon))
        time += 1
    if len(agent.memory) > batch_size:
        agent.train(batch_size) # Trenowanie agenta poprzed odtwarzanie parametrów z poprzedniego epizodu.
    if e % 50 == 0:
        agent.save(output_dir + "weights_" 
                   + '{:04d}'.format(e) + ".hdf5") 

Epizod: 0/999, ocena: 42, e: 1.0
Epizod: 1/999, ocena: 26, e: 0.99
Epizod: 2/999, ocena: 9, e: 0.99
Epizod: 3/999, ocena: 10, e: 0.99
Epizod: 4/999, ocena: 19, e: 0.98
Epizod: 5/999, ocena: 8, e: 0.98
Epizod: 6/999, ocena: 31, e: 0.97
Epizod: 7/999, ocena: 45, e: 0.97
Epizod: 8/999, ocena: 11, e: 0.96
Epizod: 9/999, ocena: 20, e: 0.96
Epizod: 10/999, ocena: 11, e: 0.95
Epizod: 11/999, ocena: 11, e: 0.95
Epizod: 12/999, ocena: 64, e: 0.94
Epizod: 13/999, ocena: 32, e: 0.94
Epizod: 14/999, ocena: 12, e: 0.93
Epizod: 15/999, ocena: 25, e: 0.93
Epizod: 16/999, ocena: 46, e: 0.92
Epizod: 17/999, ocena: 54, e: 0.92
Epizod: 18/999, ocena: 12, e: 0.91
Epizod: 19/999, ocena: 14, e: 0.91
Epizod: 20/999, ocena: 29, e: 0.9
Epizod: 21/999, ocena: 18, e: 0.9
Epizod: 22/999, ocena: 16, e: 0.9
Epizod: 23/999, ocena: 26, e: 0.89
Epizod: 24/999, ocena: 20, e: 0.89
Epizod: 25/999, ocena: 30, e: 0.88
Epizod: 26/999, ocena: 8, e: 0.88
Epizod: 27/999, ocena: 8, e: 0.87
Epizod: 28/999, ocena: 8, e: 0.87
Epiz

In [ ]:
# Zapisanych agentów można załadować za pomocą matody agent.load("./path/filename.hdf5")